<a href="https://colab.research.google.com/github/wavemx/AI_spectogram/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://mpp0xc0ae45ef.blob.core.windows.net/drivendata-mpp-storage/data/19/public/data-release.zip

!unzip -o data-release.zip

# **ALL in One**

In [0]:
import pandas as pd
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

import keras
from keras import optimizers
from keras.utils import to_categorical 
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.optimizers import rmsprop
from keras import regularizers
from keras import optimizers

#Read Data
train_df = pd.read_csv('train_labels.csv', index_col=0)

train_df['current_file'] = train_df.index.map(lambda id: f'train/{id}_c.png')
train_df['voltage_file'] = train_df.index.map(lambda id: f'train/{id}_v.png')

print(train_df.head())

#Helper Function

def read_spectograms(file_paths, img_rows, img_cols, as_gray, channels):
  """
  Reads the spectogram files from disk and normalizes the pixel values
    @params:
      file_paths - Array of file paths to read from
      img_rows - The image height.
      img_cols - The image width.
      as_grey - Read the image as Greyscale or RGB.
      channels - Number of channels.
    @returns:
      The created and compiled model (Model)        
  """
  images = []
  
  for file_path in file_paths:
        images.append(imread(file_path, as_gray = as_gray))
        
        images = np.asarray(images, dtype=np.float32)
  
  # normalize
        images = images / np.max(images)
  
  # reshape to match Keras expectaions
        images = images.reshape(images.shape[0], img_rows, img_cols, channels)
        return images

#Parameters

as_gray = True
in_channel = 4

if as_gray:
    in_channel = 1

img_rows, img_cols = 176, 128
num_classes = 11 # number of appliances

batch_size = 32
epochs = 20
input_shape = (img_rows, img_cols, in_channel)
input_img = Input(shape = input_shape)

#Current files

x_train_current = read_spectograms(train_df.current_file.values, img_rows, img_cols, as_gray, in_channel)

#Voltage files

x_train_voltage = read_spectograms(train_df.voltage_file.values, img_rows, img_cols, as_gray, in_channel)

#Labels

labels = train_df.appliance.values

# convert class vectors to binary class matrices One Hot Encoding
labels = keras.utils.to_categorical(labels, num_classes)

#Show Data
appliances = [
    'Compact Fluorescent Lamp', 
    'Hairdryer', 
    'Microwave', 
    'Air Conditioner', 
    'Fridge', 
    'Laptop', 
    'Vacuum', 
    'Incandescent Light Bulb', 
    'Fan',
    'Washing Machine',
    'Heater'
]

# pick a random index from the list
rn_appliance = np.random.choice(train_df.appliance.values)
rn_label = train_df.appliance.values[rn_appliance]
rn_current = x_train_current[rn_appliance]
rn_voltage = x_train_voltage[rn_appliance]

plt.figure()
plt.axis('off')

plt.suptitle(f"{appliances[rn_label]} (Label: {rn_label})", fontsize="x-large")

plt.subplot(121)
curr_img = None
if as_gray:
    curr_img = np.reshape(rn_current, (img_rows, img_cols))
else:
    curr_img = np.reshape(rn_current, (img_rows, img_cols, in_channel))

plt.imshow(curr_img, cmap='gray')
plt.title("Current")
plt.xticks([])
plt.yticks([])

plt.subplot(122)
if as_gray:
    curr_img = np.reshape(rn_voltage, (img_rows, img_cols))
else:
    curr_img = np.reshape(rn_voltage, (img_rows, img_cols, in_channel))

plt.imshow(curr_img, cmap='gray')
plt.title("Voltage")
plt.xticks([])
plt.yticks([])
plt.show()

#Create and train the model
#Split in train and test batches

x_train_comp = np.stack((x_train_current, x_train_voltage), axis=4)

x_train, x_test, y_train, y_test = train_test_split(x_train_comp, labels, test_size = 0.4, random_state=666)

# take them apart
x_train_current = x_train[:,:,:,:,0]
x_test_current = x_test[:,:,:,:,0]

x_train_voltage = x_train[:,:,:,:,1]
x_test_voltage = x_test[:,:,:,:,1]

#Create Model
def create_convolution_layers(input_img):
  model = Conv2D(32, (3, 3), padding='same', input_shape=input_shape)(input_img)
  model = LeakyReLU(alpha=0.1)(model)
  model = MaxPooling2D((2, 2),padding='same')(model)
  model = Dropout(0.25)(model)
  
  model = Conv2D(64, (3, 3), padding='same')(model)
  model = LeakyReLU(alpha=0.1)(model)
  model = MaxPooling2D(pool_size=(2, 2),padding='same')(model)
  model = Dropout(0.25)(model)
    
  model = Conv2D(128, (3, 3), padding='same')(model)
  model = LeakyReLU(alpha=0.1)(model)
  model = MaxPooling2D(pool_size=(2, 2),padding='same')(model)
  model = Dropout(0.4)(model)
    
  return model
current_input = Input(shape=input_shape)
current_model = create_convolution_layers(current_input)

voltage_input = Input(shape=input_shape)
voltage_model = create_convolution_layers(voltage_input)

conv = concatenate([current_model, voltage_model])

conv = Flatten()(conv)

dense = Dense(512)(conv)
dense = LeakyReLU(alpha=0.1)(dense)
dense = Dropout(0.5)(dense)

output = Dense(num_classes, activation='softmax')(dense)

model = Model(inputs=[current_input, voltage_input], outputs=[output])

opt = optimizers.Adam()

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

print(model.summary())

#Train Model

best_weights_file="weights.best.hdf5"
checkpoint = ModelCheckpoint(best_weights_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#lr_reducer = ReduceLROnPlateau(verbose=1)

callbacks = [checkpoint]

model.fit([x_train_current, x_train_voltage], y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=callbacks,
          verbose=1,
          validation_data=([x_test_current, x_test_voltage], y_test),
          shuffle=True)
          

#Basic Evaluation

# load weights
model.load_weights(best_weights_file)

final_loss, final_acc = model.evaluate([x_test_current, x_test_voltage], y_test, verbose=1)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

#Prediction

predict_df = pd.read_csv('submission_format.csv', index_col=0)

predict_df['current_file'] = predict_df.index.map(lambda id: f'test/{id}_c.png')
predict_df['voltage_file'] = predict_df.index.map(lambda id: f'test/{id}_v.png')

x_test_current = read_spectograms(predict_df.current_file.values, img_rows, img_cols, as_gray, in_channel)
x_test_voltage = read_spectograms(predict_df.voltage_file.values, img_rows, img_cols, as_gray, in_channel)


#get the predictions for the test data

predicted_classes = model.predict([x_test_current, x_test_voltage])

predict_df.appliance = np.argmax(predicted_classes,axis=1)

predict_df = predict_df.drop(['current_file', 'voltage_file'], axis=1)

predict_df.to_csv('submission.csv')

#from google.colab import files

#files.download('submission.csv')



# **NEW CODE**

In [81]:


#!/usr/bin/env python

"""
Prepares data for easier processing by the neural net classifier.
"""
import logging


import math
from io import BytesIO
from zipfile import ZipFile

import numpy as np
import pandas as pd
from PIL import Image
#from PIL import Image


__author__ = 'Greg Gasowski'
__copyright__ = 'Copyright 2019, WaveMax LLC'
__credits__ = ['Greg Gasowski']
__license__ = 'MIT'
__version__ = '1.0'
__maintainer__ = 'Greg Gasowski'
__email__ = 'gregory.gasowski@gmail.com'
__status__ = 'Production'

#Configuration/parameters
classes_count = 11
image_width = (2*118)//1
image_height = 128//1
image_data_size = image_width*image_height
channels = 1

data_root_path = './data/'

input_data_file_path = data_root_path + 'data-release.zip'

training_image_data_file_path = data_root_path + 'image_train.data'
training_labels_data_file_path = data_root_path + 'image_train_labels.csv'
testing_data_file_path = data_root_path + 'image_test.data'

testing_submission_file_path = data_root_path + 'submission_format.csv'

def compose_train_image(p_img1, p_img2) :
    """
    Creates a horizontally stacked image using two input images

    @params:
        p_img1 - Required : first image source (Image)
        p_img2 - Required : second image source (Image) 

    @returns:
        The stacked image (Image)    
    """ 

    #Stacks images horizontally (i.e. one afer another on width axis)
    img_merge_data = np.hstack([np.asarray(p_img1), np.asarray(p_img2)])
    img_merge = Image.fromarray( img_merge_data )
        
    return img_merge

def get_image_data(p_image) :
    """
    Returns a flatten array of image pixel values (1 channel gray pallete)

    @params:
        p_image - Required : the input image (Image)

    @returns:
        Flattened array of image data (array)
    """  

    #Generates image data from the received image object
    width, height = p_image.size
    data = np.asarray(p_image).reshape(height*width)
    
    return data


def create_trainining_images_data_file(p_input_data_file_path, p_training_data_file_path):
    """
    Creates training information data

    @params:
        p_input_data_file_path - Required: the input data file path (String)
        p_training_data_file_path - Required: the output training data file path (String)

    @returns:
        The extracted training labels (array)
    """  

    training_labels_file_path = 'train_labels.csv'
    
    labels = None

    with open(p_training_data_file_path, 'w+b') as data_file :
        with ZipFile(p_input_data_file_path) as data_zip:
            with data_zip.open(training_labels_file_path) as train_labels_file:
                content = train_labels_file.read()
                with BytesIO(content) as io_content:
                    train_labels = pd.read_csv(io_content)

                    max_count = train_labels.shape[0]    
                    labels = np.zeros(max_count)

                    count = 0

                    for _, row in train_labels.iterrows() :

                        with data_zip.open('train/' + str(row["id"]) + "_c.png") as c_file :
                            with BytesIO(c_file.read()) as input_buffer:
                                c_image = Image.open(input_buffer).convert("L")

                        with data_zip.open('train/' + str(row["id"]) + "_v.png") as v_file :
                            with BytesIO(v_file.read()) as input_buffer:
                                v_image = Image.open(input_buffer).convert("L")

                        image_data = get_image_data(compose_train_image(c_image, v_image))

                        labels[count] = row["appliance"]
                        data_file.write(image_data)

                        count = count + 1       

    return labels[:count]

def create_training_labels(p_labels, p_labels_data_file_path) :
    """
    Writes the training labels to a destination file

    @params:
        p_labels - Required: the array of labels (array)
    """ 

    classes = pd.DataFrame(p_labels.astype(int))
    classes.to_csv(p_labels_data_file_path, header=None)

    return


def create_testing_images_data_file(p_input_data_file_path, p_testing_data_file_path):
    """
    Creates testing information data

    @params:
        p_input_data_file_path - Required: the input data file path (String)
        p_testing_data_file_path - Required: the output testing data file path (String)

    @returns:
        The count of test images (int)
    """  

    submission_format_file_path = 'submission_format.csv'

    with open(p_testing_data_file_path, 'w+b') as data_file :
        with ZipFile(p_input_data_file_path) as data_zip:
            with data_zip.open(submission_format_file_path) as submission_format_file:
                content = submission_format_file.read()
                with BytesIO(content) as io_content:
                    submission_indexes = pd.read_csv(io_content)

                    count = 0

                    for _, row in submission_indexes.iterrows() :

                        with data_zip.open('test/' + str(row["id"]) + "_c.png") as c_file :
                            with BytesIO(c_file.read()) as input_buffer:
                                c_image = Image.open(input_buffer).convert("L")

                        with data_zip.open('test/' + str(row["id"]) + "_v.png") as v_file :
                            with BytesIO(v_file.read()) as input_buffer:
                                v_image = Image.open(input_buffer).convert("L")

                        image_data = get_image_data(compose_train_image(c_image, v_image))
                        data_file.write(image_data)

                        count = count + 1       

    return count

def create_testing_submission(p_input_data_file_path, p_testing_submission_file_path) :
    """
    Writes the submission data to a destination file

    @params:
        p_input_data_file_path - Required: the input data file path (String)
        p_testing_submission_file_path - Required: the testing submission file path (String)
    """ 
    submission_format_file_path = 'submission_format.csv'

    with ZipFile(p_input_data_file_path) as data_zip:
        with data_zip.open(submission_format_file_path) as submission_format_file:
            content = submission_format_file.read()
            with BytesIO(content) as io_content:
                submission_indexes = pd.read_csv(io_content)
                submission_indexes.to_csv(p_testing_submission_file_path, index=False)

    return

def main() :
    """
    Entry point
    """
    logging.basicConfig(level=logging.INFO)
    
    #create training data
    logging.info('Creating training data ...')
    training_labels  = create_trainining_images_data_file(input_data_file_path, training_image_data_file_path)
    create_training_labels(training_labels, training_labels_data_file_path)
    logging.info("Processed training images count: %d" % training_labels.shape[0])
    logging.info('Creating training data DONE')


    logging.info('Creating testing data ...')
    testing_count = create_testing_images_data_file(input_data_file_path, testing_data_file_path)
    create_testing_submission(input_data_file_path, testing_submission_file_path)
    logging.info("Processed testing images count: %d" % testing_count)
    logging.info('Creating testing data DONE')

if __name__ == '__main__':
    main()

INFO:root:Creating training data ...
INFO:root:Processed training images count: 576
INFO:root:Creating training data DONE
INFO:root:Creating testing data ...
INFO:root:Processed testing images count: 384
INFO:root:Creating testing data DONE


In [0]:
"""
Classifies the test data and generates the submissions.
"""
import logging


import numpy as np
import numpy.random as rnd
import pandas as pd
import time
import matplotlib.pyplot as plt
import math
import keras.utils.np_utils as utl

from sklearn.preprocessing import LabelBinarizer

from PIL import Image, ImageFilter
from tensorflow import set_random_seed
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.optimizers import rmsprop
from keras import regularizers
from keras import optimizers

In [0]:

__author__ = 'Greg Gasowski'
__copyright__ = 'Copyright 2019, WaveMax LLC'
__credits__ = ['Greg Gasowski']
__license__ = 'MIT'
__version__ = '1.0'
__maintainer__ = 'Greg Gasowski'
__email__ = 'gregory.gasowski@gmail.com'
__status__ = 'Production'

#configuration data 
image_height = 128//1
image_width = 176//1
#image_width = 2*118//1
training_image_count = 576
testing_image_count = 384
classes_count = 11

data_root_path = './data/'

training_image_data_file_path = data_root_path + 'image_train.data'
training_labels_data_file_path = data_root_path + 'image_train_labels.csv'
testing_data_file_path = data_root_path + 'image_test.data'

testing_submission_file_path = data_root_path + 'submission_format.csv'
submission_results_file_path =  data_root_path + 'submission_results.csv'


In [0]:
def read_image(p_image_data_file_path, p_position, p_image_width, p_image_height) :
    """
    Reads an image from an image data from a image data repository @see prepare_data.py

    @params:
        p_image_data_file_path - Required : the image data file path (String)
        p_position - Required : second image source (int)
        p_image_width - Required: the image width (int)
        p_image_height - Required: the image height (int)
    
    @returns:
        The image data (array)    
    """  
    with open(p_image_data_file_path, "rb") as image_file :
        image_file.seek(p_position * p_image_height* p_image_width)
        data = image_file.read(p_image_height * p_image_width)
    
        data_b = np.frombuffer(data, dtype=np.uint8)

    return np.asarray(data_b)


In [0]:
def process_images(p_images, p_image_width, p_image_height) :
    """
    Processes a set of images so it can be classified by the neurals network model

    @params:
        p_images - Required : the images to process (String)
        p_image_width - Required: the image width (int)
        p_image_height - Required: the image height (int)
    """  
    #reshape according to inputs accepted by a Conv2d layer
    processed_images = p_images.reshape(p_images.shape[0], p_image_height, p_image_width, 1)

    #data normalization to max value (0-255 grayscale values)
    processed_images = (processed_images * 1.0) /255
 
    return processed_images
  

In [0]:
def read_labels(p_labels_file_path) :
    """
    Reads the extracted training labels @see prepare_data.py

    @params:
        p_labels_file_path - Required : the data file path (String)
    @returns:
        A dataframe containing the read labels with the column [id] for ordinal id and [label] for the label value    
    """ 
    labels = pd.read_csv(p_labels_file_path, header= None)
    labels.columns = ["id", "label"]
  
    return labels


In [0]:
def process_labels(p_labels) :
    """
    Processes the read labels

    @params:
        p_labels - Required: the read labels (array)
    @returns:
        The processed labels (binarization - one hot-encoded)    
    """
    processed_labels = LabelBinarizer().fit_transform(p_labels)
    
    return processed_labels


In [0]:
   def generate_train_set(
                            p_image_training_data_file_path, 
                            p_labels_file_path, 
                            p_train_set_size, 
                            p_image_width, 
                            p_image_height
                        ) :
                            """
                            Generates the training data set

                            @params:
                                p_image_training_data_file_path - Required: the training image data file path (String)
                                p_labels_file_path - Required: the labels file path (String)
                                p_train_set_size - Required: the size of the training set (int)
                                p_image_width - Required: the image width (int)
                                p_image_height - Required: the image height (int)

                            @returns:
                                (train_labels_processed, train_images_processed) tuple wiht the the processed train labels (array) 
                                and the processed train images (array)
                            """
                            labels = read_labels(p_labels_file_path)

                            labels_batch = np.zeros(p_train_set_size)
                            labels_batch = labels["label"][0:p_train_set_size].values

                            images_batch = []

                            for i in range(0, p_train_set_size) :
                                image_data = read_image(p_image_training_data_file_path, i, p_image_width, p_image_height)
                                images_batch.append(image_data.reshape(p_image_height, p_image_width))
                                #images_batch.append(image_data.reshape(p_image_height, p_image_width))

                            train_labels_processed = process_labels(labels_batch)

                            train_images_processed = process_images(np.array(images_batch), p_image_width, p_image_height)

                            return train_labels_processed, train_images_processed


In [0]:
def generate_test_set(
    p_test_image_data_file_path, 
    p_test_set_size, 
    p_image_width, 
    p_image_height
) :
    """
    Generates the test data set

    @params:
        p_test_image_data_file_path - Required: the testing image data file path (String)
        p_test_set_size - Required: the size of the testing set (int)
        p_image_width - Required: the image width (int)
        p_image_height - Required: the image height (int)

    @returns:
        test_images_processed the processed test images (array)
    """
    images_batch = []

    for i in range(0, p_test_set_size) :
        image_data = read_image(p_test_image_data_file_path, i, p_image_width, p_image_height)
        images_batch.append(image_data.reshape(p_image_height, p_image_width))

    test_images_processed = process_images(np.array(images_batch), p_image_width, p_image_height)

    return test_images_processed  

In [0]:
def create_model(p_image_width, p_image_height, p_num_classes) :
    """
    Creates the compiled model for image classification.

    @params:
        p_image_width - Required: the image width (int)
        p_image_height - Required: the image height (int)
        p_num_classes - Required: the number of classes

    @returns:
      The created and compiled model (Model)        
    """
    
    input_shape = (p_image_height, p_image_width, 1)

    #we will use a sequential model for training 
    model = Sequential()
    #CONV 3x3x32 => RELU => NORMALIZATION => MAX POOL 3x3 block
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=input_shape))
    model.add(Activation("relu"))
    #model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))
    

    #CONV 3x3x64 => RELU => NORMALIZATION => MAX POOL 2x2 block
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    #model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    #CONV 3x3x128 => RELU => NORMALIZATION => MAX POOL 2x2 block
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    #kernel_regularizer=regularizers.l2(300)
    #model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

     #CONV 3x3x256 => RELU => NORMALIZATION => MAX POOL 2x2 added on 12/08
    #model.add(Conv2D(256, (3, 3), padding="same"))
    #model.add(Activation("relu"))
    #model.add(ZeroPadding2D((1,1)))
    #model.add(BatchNormalization(axis=-1))
    #model.add(MaxPooling2D((2, 2), strides=(2,2)))
    #model.add(Dropout(0.25))

    #FLATTEN => DENSE 1024 => RELU => NORMALIZATION block
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    #model.add(BatchNormalization())
    model.add(Dropout(0.5))

    #final DENSE => SOFTMAX block for multi-label classification
    model.add(Dense(p_num_classes))
    model.add(Activation("softmax"))

    model.summary()

    #using categorical_crossentropy loss function with adam optimizer
    #model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"]) 

    return model

In [0]:
def train_model(
    p_model, 
    p_training_image_data, 
    p_trainging_labels, 
    p_batch_size = 32, 
    p_epochs_to_train = 80, 
    p_verbose_level = 2
) :
    """
    Trains the model using the train image data and train labels.
    
    @parameters:
      p_model - Required: the Keras model to be trained (Model)
      p_training_image_data - Required: the image data used for training (array)
      p_training_labels - Required: the training labels used fo training (array)
      p_batch_size - Optional, default 32: the batch size used for training (int)
      p_epochs_to_train - Optional, default 50: number of training epochs (int)
      p_verbose_level - Optional, default 2: the Keras verbose level (int)
    
    @returns:
      The trained model (Model)
    """    
    p_model.fit(
        x = p_training_image_data, 
        y = p_trainging_labels, 
        batch_size = p_batch_size, 
        epochs = p_epochs_to_train,
        shuffle = True,
        verbose = p_verbose_level    
    )
    
    return p_model


In [0]:
def predict_labels(p_model, p_test_image_data, p_batch_size = 32) :
    """
    Predicts the labels associated with the test data.
    
    @parameters:
      p_model - Required: the Keras model to be used (Model)
      p_test_image_data - Required: the image data used for testing (array)
      p_batch_size - Optional, default 32: the batch size used for training (int)
    
    @returns:
      The predicted label (array)
    """      
    labels = p_model.predict_classes(p_test_image_data, p_batch_size)
  
    return labels

In [0]:
def write_results(
    p_testing_submission_file_path, 
    p_submission_results_file_path, 
    p_results
) :
    """
    Writes the result to the output file.
    
    @parameters:
      p_testing_submission_file_path - Required: the path to the submission format (String)
      p_submission_results_file_path - Required: the path to the output file (String)
      p_results - Required: the results to be written in the outut file (array)
    """     
    submission_structure = pd.read_csv(p_testing_submission_file_path)
    submission_structure['appliance'] = p_results
    submission_structure.to_csv(p_submission_results_file_path, index=False)
  

In [0]:
def main():
    logging.basicConfig(level=logging.INFO)
    
    #prepare training data
    logging.info('Reading training data ...')
    train_labels, train_images = generate_train_set(
        training_image_data_file_path, 
        training_labels_data_file_path, 
        training_image_count, 
        image_width, 
        image_height
    )
    logging.info('Reading training data DONE')
    
    #create and train model
    logging.info('Creating model ...')
    model = create_model (image_width, image_height, classes_count)
    logging.info('Creating model DONE')

    logging.info('Training model ... ')
    model = train_model(model, train_images, train_labels, p_epochs_to_train = 80)
    logging.info('Training model DONE')
    
    #create test data
    logging.info('Reading testing data ...')
    test_images = generate_test_set(
      testing_data_file_path, 
      testing_image_count, 
      image_width, 
      image_height
    )
    logging.info('Reading testing data DONE')
    
    #predict labels for test data
    logging.info('Predicting test data classes ...')
    result = predict_labels(model, test_images)
    logging.info('Predicting test data classes DONE')
    
    #write results
    logging.info('Writing results ...')
    write_results(
        testing_submission_file_path, 
        submission_results_file_path, 
        result
    )
    logging.info('Writing results DONE')
    
    
if __name__ == '__main__':
    main()

INFO:root:Reading training data ...
INFO:root:Reading training data DONE
INFO:root:Creating model ...
INFO:root:Creating model DONE
INFO:root:Training model ... 


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_69 (Conv2D)           (None, 128, 176, 32)      320       
_________________________________________________________________
activation_35 (Activation)   (None, 128, 176, 32)      0         
_________________________________________________________________
max_pooling2d_69 (MaxPooling (None, 42, 58, 32)        0         
_________________________________________________________________
dropout_53 (Dropout)         (None, 42, 58, 32)        0         
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 42, 58, 64)        18496     
_________________________________________________________________
activation_36 (Activation)   (None, 42, 58, 64)        0         
_________________________________________________________________
max_pooling2d_70 (MaxPooling (None, 21, 29, 64)       

In [0]:
 
from google.colab import files

files.download('./data/submission_results.csv')

In [0]:
 #plot data
    plt.plot(model['train_model'], label='Training')
    plt.plot(test_images['generate_test_set'], label='Testing')
    plt.ylabel('loss')
    plt.xlabel('iteration ')  
    plt.title('training loss iterations')
    plt.legend()